In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# A very basic SAE Training Tutorial

Please note that it is very easy for tutorial code to go stale so please have a low bar for raising an issue in the 

## Setup

In [2]:
import torch
import os
import sys

sys.path.append("..")

from sae_lens.training.config import LanguageModelSAERunnerConfig
from sae_lens.training.lm_runner import language_model_sae_runner

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


# Model Selection and Evaluation (Feel Free to Skip)

We'll use the runner to train an SAE on a TinyStories Model. This is a very small model so we can train an SAE on it quite quickly. Before we get started, let's load in the model with `transformer_lens` and see what it can do. 

TransformerLens gives us 2 functions that are useful here (and circuits viz provides a third):
1. `transformer_lens.utils.test_prompt` will help us see when the model can infer one token.
2. `HookedTransformer.generate` will help us see what happens when we sample from the model.
3. `circuitsvis.logits.token_log_probs` will help us visualize the log probs of tokens at several positions in a prompt.

In [3]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained(
    "tiny-stories-1L-21M"
)  # This will wrap huggingface models and has lots of nice utilities.

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

/home/hufy/SAELens/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


### Getting a vibe for a model using `model.generate`

Let's start by generating some stories using the model.

In [4]:
# here we use generate to get 10 completeions with temperature 1. Feel free to play with the prompt to make it more interesting.
for i in range(5):
    display(
        model.generate(
            "Once upon a time",
            stop_at_eos=False,  # avoids a bug on MPS
            temperature=1,
            verbose=False,
            max_new_tokens=50,
        )
    )

'Once upon a time, there was a bear named Bill. He had two bald brothers - Sam and Max. Max was not very fluffy and he was often scared of something that was just his own.\nOne day, Jack gave his brother were eating and he noticed that'

'Once upon a time, there was an incredible angel. Sometimes the angel would talk to her. One day the angel was playing with her friends.\n\nThe angel said, "Don\'t worry. You will love to protect the children. Angels are around here for to'

'Once upon a time, there was a lonely hero who had been on a quest. He was looking for someone to marry, no one.\n\nHe lived in the hot castle. He looked outside to see a little woman, and he knew she must stay indoors.'

'Once upon a time, there were two friends, Bob and Sam. They were Sam, who always shared and were the same. The old man often refused this time.\n\nOne day, Bob got a fight with another man. This man was commanded for his family'

"Once upon a time, there was a wet forest. Inside her temple was a pretty picture. She wanted to fix the temple, but was scared she wouldn't leave. So, when the temple was dry, she grabbed an umbrella and went to replace it. She wiped"

One thing we notice is that the model seems to be able to repeat the name of the main character very consistently. It can output a pronoun intead but in some stories will repeat the protagonists name. This seems like an interesting capability to analyse with SAEs. To better understand the models ability to remember the protagonists name, let's extract a prompt where the next character is determined and use the "test_prompt" utility from TransformerLens to check the ranking of the token for that name.

### Spot checking model abilities with `transformer_lens.utils.test_prompt`

In [5]:
from transformer_lens.utils import test_prompt

# Test the model with a prompt
test_prompt(
    "Once upon a time, there was a little girl named Lily. She lived in a big, happy little girl. On her big adventure,",
    " Lily",
    model,
    prepend_space_to_answer=False,
)

Tokenized prompt: ['<|endoftext|>', 'Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' little', ' girl', ' named', ' Lily', '.', ' She', ' lived', ' in', ' a', ' big', ',', ' happy', ' little', ' girl', '.', ' On', ' her', ' big', ' adventure', ',']
Tokenized answer: [' Lily']


Performance on answer token:
Rank: 1        Logit: 18.81 Prob: 13.46% Token: | Lily|

Top 0th token. Logit: 20.48 Prob: 71.06% Token: | she|
Top 1th token. Logit: 18.81 Prob: 13.46% Token: | Lily|
Top 2th token. Logit: 17.35 Prob:  3.11% Token: | the|
Top 3th token. Logit: 17.26 Prob:  2.86% Token: | her|
Top 4th token. Logit: 16.74 Prob:  1.70% Token: | there|
Top 5th token. Logit: 16.43 Prob:  1.25% Token: | they|
Top 6th token. Logit: 15.80 Prob:  0.66% Token: | all|
Top 7th token. Logit: 15.64 Prob:  0.56% Token: | things|
Top 8th token. Logit: 15.28 Prob:  0.39% Token: | one|
Top 9th token. Logit: 15.24 Prob:  0.38% Token: | lived|


Ranks of the answer tokens: [(' Lily', 1)]

In the output above, we see that the model assigns ~ 70% probability to "she" being the next token, and a 13% chance to " Lily" being the next token. Other names like Lucy or Anna are not highly ranked. 

### Exploring Model Capabilities with Log Probs

Looking at token ranking for a single prompt is interesting, but a much higher through way to understand models is to look at token log probs for all tokens in text. We can use the `circuits_vis` package to get a nice visualization where we can see tokenization, and hover to get the top5 tokens by log probability. Darker tokens are tokens where the model assigned a higher probability to the actual next token.

In [6]:
import circuitsvis as cv  # optional dep, install with pip install circuitsvis

# Let's make a longer prompt and see the log probabilities of the tokens
example_prompt = """Hi, how are you doing this? I'm really enjoying your posts"""
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)
# hover on the output to see the result.

ModuleNotFoundError: No module named 'circuitsvis'

Let's combine `model.generate` and the token log probs visualization to see the log probs on text generated by the model. Note that we can play with the temperature and this should sample less likely trajectories according to the model. I've increased the maximum number of tokens in order to get a full story.

Some things to explore:
- Which tokens does the model assign high probability to? Can you see how the model should know which word comes next?
- What happens if you increase / decrease the temperature?
- Do the rankings of tokens seem sensible to you? What about where the model doesn't assign a high probability to the token which came next?

In [ ]:
example_prompt = model.generate(
    "Once upon a time",
    stop_at_eos=False,  # avoids a bug on MPS
    temperature=1,
    verbose=True,
    max_new_tokens=200,
)
logits, cache = model.run_with_cache(example_prompt)
cv.logits.token_log_probs(
    model.to_tokens(example_prompt),
    model(example_prompt)[0].log_softmax(dim=-1),
    model.to_string,
)

# Training an SAE

Now we're ready to train out SAE. We'll make a runner config, instantiate the runner and the rest is taken care of for us!

During training, you use weights and biases to check key metrics which indicate how well we are able to optimize the variables we care about.

To get a better sense of which variables to look at, you can read my (Joseph's) post [here](https://www.lesswrong.com/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream) and especially look at my weights and biases report [here](https://links-cdn.wandb.ai/wandb-public-images/links/jbloom/uue9i416.html).

A few tips:
- Feel free to reorganize your wandb dashboard to put L0, CE_Loss_score, explained variance and other key metrics in one section at the top.
- Make a [run comparer](https://docs.wandb.ai/guides/app/features/panels/run-comparer) when tuning hyperparameters.
- You can download the resulting sparse autoencoder / sparsity estimate from wandb and upload them to huggingface if you want to share your SAE with other.
    - cfg.json (training config)
    - sae_weight.safetensors (model weights)
    - sparsity.safetensors (sparsity estimate)

## MLP Out

I've tuned the hyperparameters below for a decent SAE which achieves 93% CE Loss recovered and an L0 of ~60, and runs in about 22 minutes on an A100 (sorry peeps, that is the kinda hardware you'll want to be using moving on). I think you could hyperparameter tune toward a much better SAE in the future (and would welcome PR's that improve on the hyperparameters here!). 

In [4]:
cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="tiny-stories-1L-21M",  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    hook_point="blocks.0.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_point_layer=0,  # Only one layer in the model.
    d_in=1024,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="geometric_median",  # The geometric median can be used to initialize the decoder weights.
    # Training Parameters
    lr=0.0008,  # lower the better, we'll go fairly high to speed up the tutorial.
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=10000,  # this can help avoid too many dead features initially.
    l1_coefficient=0.001,  # will control how sparse the feature activations are
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size=4096,
    context_size=512,  # will control the lenght of the prompts we feed to the model. Larger is better but slower.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    total_training_tokens=1_000_000
    * 50,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size=16,
    # Resampling protocol
    use_ghost_grads=False,
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project="sae_lens_tutorial",
    wandb_log_frequency=10,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype=torch.float32,
)

# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder_dictionary = language_model_sae_runner(cfg)

Run name: 16384-L1-0.001-LR-0.0008-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 12207
Total wandb updates: 1220
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 12 times.
Number tokens in sparsity calculation window: 4.10e+06


/home/hufy/SAELens/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loaded pretrained model tiny-stories-1L-21M into HookedTransformer
Moving model to device:  cuda
Run name: 16384-L1-0.001-LR-0.0008-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 12207
Total wandb updates: 1220
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 12 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 16384-L1-0.001-LR-0.0008-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 12207
Total wandb updates: 1220
n_tokens_per_feature_sampling_window (millions): 2097.152
n_tokens_per_dead_feature_window (millions): 2097.152
We will reset the sparsity calculation 12 times.
Number tokens in sparsity calculation window: 4.10e+06


details/current_learning_rate_layer0,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer1,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer10,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer11,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer2,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer3,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer4,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer5,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer6,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer7,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
details/current_learning_rate_layer8,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


Objective value: 3489830.0000:   2%|▏         | 2/100 [00:00<00:02, 36.56it/s]
/home/hufy/SAELens/sae_lens/training/sparse_autoencoder.py:198: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(origin, dtype=self.dtype, device=self.device)


KeyboardInterrupt: 

## Training with GPT2

In [3]:
cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="gpt2",  # GPT2-small model
    hook_point="blocks.{layer}.hook_mlp_out",  # MLP output of each layer
    hook_point_layer=[int(i) for i in range(12)],  # All layers
    d_in=768,  # the width of the mlp output.
    dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    is_dataset_tokenized=True,
    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="geometric_median",  # The geometric median can be used to initialize the decoder weights.
    decoder_orthogonal_init = True, # Orthogonal initialization of the decoder weights
    sae_type = "tanh_sae", # Use tanh activation for the SAE
    noise_scale = 1, # Add noise to the input for tanh SAE to avoid activation collapse
    # Training Parameters
    lr=0.0001,  # lower the better, we'll go fairly high to speed up the tutorial.
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=10000,  # this can help avoid too many dead features initially.
    l1_coefficient=0.0001,  # will control how sparse the feature activations are
    l1_loss_normalization="l2_squared", # L2 squared normalization for the L1 loss
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    train_batch_size=2048,
    context_size=256,  # will control the lenght of the prompts we feed to the model. Larger is better but slower.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    total_training_tokens=1_000_000
    * 50,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    store_batch_size=16,
    # Resampling protocol
    use_ghost_grads=False,
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.
    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    wandb_project="sae_lens_tutorial",
    wandb_log_frequency=10,
    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype=torch.float32,
)

# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder_dictionary = language_model_sae_runner(cfg)

Run name: 12288-L1-1e-05-LR-0.0001-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.262144
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 24414
Total wandb updates: 2441
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 24 times.
Number tokens in sparsity calculation window: 2.05e+06
Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda
Run name: 12288-L1-1e-05-LR-0.0001-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.262144
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 24414
Total wandb updates: 2441
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 24 times.
Number tokens in sparsity calculation window: 2.05e+06
Run name: 12288-L1-1e-05-LR-0.0001-Tokens-5.000e+07
n_tokens_per_buffer (millions): 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hufy-dev. Use `wandb login --relogin` to force relogin


Objective value: 2785086.5000:   4%|▍         | 4/100 [00:00<00:00, 127.95it/s]
/home/hufy/SAELens/sae_lens/training/sparse_autoencoder.py:198: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(origin, dtype=self.dtype, device=self.device)
Objective value: 10393808.0000:   4%|▍         | 4/100 [00:00<00:00, 118.01it/s]
99| MSE Loss 3.578 | L1 155.191:   0%|          | 202752/50000000 [00:29<1:36:20, 8615.41it/s]

RuntimeError: The size of tensor a (12288) must match the size of tensor b (256) at non-singleton dimension 1

99| MSE Loss 3.578 | L1 155.191:   0%|          | 202752/50000000 [00:42<1:36:20, 8615.41it/s]

# TO DO: Understanding TinyStories-1L with our SAE

I haven't had time yet to complete this section, but I'd love to see a PR where someones uses an SAE they trained in this tutorial to understand this model better.

In [59]:
import pandas as pd

# Let's start by getting the top 10 logits for each feature

sparse_autoencoder = next(iter(sparse_autoencoder_dictionary))[1]
projection_onto_unembed = sparse_autoencoder.W_dec @ model.W_U


# get the top 10 logits.
vals, inds = torch.topk(projection_onto_unembed, 10, dim=1)

# get 10 random features
random_indices = torch.randint(0, projection_onto_unembed.shape[0], (10,))

# Show the top 10 logits promoted by those features
top_10_logits_df = pd.DataFrame(
    [model.to_str_tokens(i) for i in inds[random_indices]],
    index=random_indices.tolist(),
).T
top_10_logits_df

,16211,9873,7694,13782,9370,7699,9477,7443,227,6926
0,checking,Restore,cere,In,speakers,ju,32,ow,bum,owner
1,Berry,telling,uc,Kn,roller,Boot,insert,o,"?!""",clerk
2,Squ,ruce,lesson,grand,moss,drunk,improve,stop,simply,window
3,reason,observ,debate,Today,escal,upstream,xious,e,sheet,keeper
4,Bro,erc,minor,Jay,mac,",'",Todd,ah,Charlotte,owner
5,merge,gracious,mentioned,Finally,hy,traffic,Mor,you,double,keepers
6,Las,self,succeeding,Then,bumper,Soldier,allergy,wo,Joyce,worker
7,angling,regretted,national,Family,salad,pitched,Phillip,hey,fantastic,manager
8,slid,sense,fines,During,hooks,narrator,itability,whe,pron,employee
9,blast,restoration,ul,H,rings,vegetarian,aback,Ow,Dahl,employees
